# The File System

As we learned in Chapter 2 of [The Theory and Craft of Digital Preservation](https://jhupbooks.press.jhu.edu/title/theory-and-craft-digital-preservation) the *file* and *file formats* are foundational concepts for digital curation, and for computing in general.

> File formats enable most modern computing. A file format is a convention that established the rules of how information is structured and stored in a file. File extensions (.mp3, .jpg, .doc), in part, define the file and enable it to be interpreted. (p. 47)

We will be taking a closer look at some of these formats in the coming weeks. But first we are going to review some of what you learned in Introduction to Programming about how to interact with files and [file systems](https://en.wikipedia.org/wiki/File_system). While there are many types of file systems with different properties many of them offer the same Application Programming Interface or [API](https://en.wikipedia.org/wiki/API) which allows you to interact with files using read and write operations.

Digital curation practices often involve interacting with the file system in order to read, write and update files. Most computer operating systems (Windows, OS X, etc) provide access to the file system using applications like the File Explorer (Windows) and the Finder (OS X). Digital curation regularly requires *documenting* and performing *repeatable* interactions with files and the file system. Programming languages like Python are useful for automating digital curation tasks and workflows, and for documenting the curatorial behaviors. Being able to create and modify these programs is a useful digital curation skill.

This notebook provides some examples of reading from and writing to the filesystem and then asks you to perform a similar task.

## Paths

Python's [pathlib](https://docs.python.org/3/library/pathlib.html) module provides an object-oriented way to interact with the filesystem. It is named *pathlib* because it is organized around the idea of *file paths* which are locations for files on the file system. Normally you see these using a notation something like:

    /home/ed/documents/resume.pdf
    
or on Windows:

    C:\Users\Ed\Documents\resume.pdf
    
The two different notations are expressing the same thing, a location for a file on the file system by starting at the *root* of the file system and then navigating down through *folders* (also known as *directories*) until there is a name for a *file*. This is known as an *absolute path* because it is anchored to the root of the filesystem.

So for example we can crate a path for the computer's root file system:

In [8]:
import pathlib

root = pathlib.Path('/')
print(root)

/


Now that we have a `Path` instance stored in the variable named `root` we can list the contents of this path location using its [iterdir](https://docs.python.org/3/library/pathlib.html#pathlib.Path.iterdir) method in a little loop:

In [9]:
for p in root.iterdir():
    print(p)

/tmp
/boot
/bin
/cdrom
/snap
/usr
/sbin
/proc
/var
/lib
/opt
/lib32
/srv
/lost+found
/sys
/lib64
/dev
/etc
/media
/swapfile
/libx32
/run
/home
/mnt
/root


This may look different when you run it on your computer, since it may have a different root directory.

Paths can also be *relative*. Relative paths are not anchored to the root of the file system. Instead they refer to a location that is relative to program's *current working directory*. You can think of this as the home for your program, or where it was started from. Relative paths look something like this:

    ed/resume.pdf

Most programming languages let you determine the current working directory. Python's [os](https://docs.python.org/3/library/os.html) module has a function called [getcwd](https://docs.python.org/3/library/os.html?highlight=os#os.getcwd):

In [11]:
import os

os.getcwd()

'/home/ed/Projects/inst341/modules/module-02'

## Reading Files

Digital curation is fundamentally concerned with *the care of data*, and data is almost always stored in files of some kind. When caring for data we often want to *read* the files and folders on the file system to determine the formats that the data is expressed in, its size, and file fixity. We'll be discussing some of these operations in future modules. For now we are going to focus on reading the files in a directory and figuring out how much data they contain as an example

In order to make this more interesting we're going to work with a set of files from [Digital Corpora](https://digitalcorpora.org/corpora/files) which is a project to provide materials for digital forensics education. I've put a zip file of a few files on Google Drive for use to use which you can copy here and unzip. We will be talking about cloud services like Google Drive and container formats like ZIP in the coming weeks. So for now just run these notebook cells even if you don't understand them just yet.

In [14]:
! unzip inst341.zip

unzip:  cannot find or open inst341.zip, inst341.zip.zip or inst341.zip.ZIP.


Ok let's create a `Path` object for the directory we just created. We can use a relative path since the zip file unpacked into our current working directory:

In [17]:
data = pathlib.Path('inst341')
print(data)

inst341


We can list the files that were unpacked from the ZIP file:

In [19]:
for p in data.iterdir():
    print(p)

inst341/761213.ps
inst341/441236.ps
inst341/763624.html
inst341/951980.gz
inst341/033333.pdf
inst341/115389.jpg
inst341/377087.txt
inst341/306840.html
inst341/368751.html
inst341/362088.pdf
inst341/215842.txt
inst341/120637.ppt
inst341/481368.csv
inst341/512608.ppt
inst341/710097.pdf
inst341/116114.dbase3
inst341/286538.pdf
inst341/064568.pdf
inst341/789265.ppt
inst341/925740.doc
inst341/447656.html
inst341/278141.jpg
inst341/356028.pdf
inst341/098807.doc
inst341/837467.jpg


So it looks like a collection of different types of files (pdf, doc, jpg, etc). If we want we can try to read one of them in. Take the `inst341/377087.txt` as an example. We can create a `Path` object for it and then use the `read_text` method to read its contents into a variable that we can print out:

In [23]:
p = pathlib.Path('inst341/377087.txt')
contents = p.read_text()
print(contents)


000
FONT12 KNHC 100237
PWSAT2
TROPICAL DEPRESSION PALOMA WIND SPEED PROBABILITIES NUMBER  19      
NWS TPC/NATIONAL HURRICANE CENTER MIAMI FL   AL172008               
0300 UTC MON NOV 10 2008                                            
                                                                    
AT 0300Z THE CENTER OF TROPICAL DEPRESSION PALOMA WAS LOCATED NEAR  
LATITUDE 22.0 NORTH...LONGITUDE 78.0 WEST WITH MAXIMUM SUSTAINED    
WINDS NEAR 25 KTS...30 MPH...45 KM/HR.                              
                                                                    
Z INDICATES COORDINATED UNIVERSAL TIME (GREENWICH)                  
   ATLANTIC STANDARD TIME (AST)...SUBTRACT 4 HOURS FROM Z TIME      
   EASTERN  STANDARD TIME (EST)...SUBTRACT 5 HOURS FROM Z TIME      
   CENTRAL  STANDARD TIME (CST)...SUBTRACT 6 HOURS FROM Z TIME      
                                                                    
                                                                    
I. 

This is a somewhat random assortment of files, so perhaps it isn't the best example. But this looks like this text file contains a weather report for a tropical storm Paloma in 2008?

## File Sizes and Storage

Perhaps a more practical task would be to determine how much storage space these files take up? Knowing how much storage different types of data take up is often very important for digital curation tasks because storage space is expensive over time. Some files can be compressed to save space. Some files may not be worth saving if they are available in compressed formats. Being able to programatically ask questions of the our data can be veyr helpful.

A `Path` object has a [stat](https://docs.python.org/3/library/pathlib.html#pathlib.Path.stat) method which returns information about the file, such as the size. Here's the size in bytes of the weather report we just looked at:

In [25]:
p = pathlib.Path('inst341/377087.txt')
info = p.stat()
print(info.st_size)

5817


Or we could print out the sizes for all the files in the directory:

In [29]:
data = pathlib.Path('inst341')

for p in data.iterdir():
    info = p.stat()
    print(p, info.st_size)

inst341/761213.ps 11196
inst341/441236.ps 57926
inst341/763624.html 28650
inst341/951980.gz 205652
inst341/033333.pdf 223686
inst341/115389.jpg 71167
inst341/377087.txt 5817
inst341/306840.html 34040
inst341/368751.html 11267
inst341/362088.pdf 3555072
inst341/215842.txt 1244
inst341/120637.ppt 59392
inst341/481368.csv 36096
inst341/512608.ppt 33792
inst341/710097.pdf 42019
inst341/116114.dbase3 11394
inst341/286538.pdf 133799
inst341/064568.pdf 641356
inst341/789265.ppt 5257728
inst341/925740.doc 34816
inst341/447656.html 12258
inst341/278141.jpg 39479
inst341/356028.pdf 142229
inst341/098807.doc 755200
inst341/837467.jpg 6654


We could also figure out the size in bytes of all of the files:

In [30]:
data = pathlib.Path('inst341')
total_size = 0

for p in data.iterdir():
    info = p.stat()
    total_size += info.st_size
    
print(total_size)

11411929


Or to get a bit little bit more fancy we can use a Python [Counter](https://docs.python.org/3/library/collections.html?highlight=counter#collections.Counter) we could count up how much space the files take up by file extension:

In [33]:
import collections

sizes = collections.Counter()

for p in data.iterdir():
    info = p.stat()
    size = info.st_size
    
    extension = p.suffix
    sizes[extension] += size
    
for extension, size in sizes.most_common():
    print(extension, size)

.ppt 5350912
.pdf 4738161
.doc 790016
.gz 205652
.jpg 117300
.html 86215
.ps 69122
.csv 36096
.dbase3 11394
.txt 7061


It looks like the .ppt (Powerpoint) and .pdf (Portable Document Format) files take up the majority of the space.

## Exercise

There is a file with your username.zip 